In [ ]:
# Install all project dependencies
!pip install -r requirements.txt

# Import necessary libraries
import os
import base64
from kaggle_secrets import UserSecretsClient

# --- 1. Set up SSH for pushing to GitHub ---
print("--- Setting up SSH for GitHub ---")
user_secrets = UserSecretsClient()
git_key_b64 = user_secrets.get_secret("GIT_DEPLOY_KEY_B64")

ssh_dir = "/root/.ssh"
os.makedirs(ssh_dir, exist_ok=True)
key_path = os.path.join(ssh_dir, "id_ed25519")
with open(key_path, "w") as f:
    f.write(base64.b64decode(git_key_b64).decode())
os.chmod(key_path, 0o600)
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# --- 2. Set up DVC Credentials ---
print("\n--- Setting up DVC Credentials ---")
gdrive_creds_b64 = user_secrets.get_secret("GDRIVE_CREDENTIALS_DATA_B64")
with open("gdrive-credentials.json", "w") as f:
    f.write(base64.b64decode(gdrive_creds_b64).decode())

# --- 3. Configure Git ---
!git config --global user.name "Kaggle CI Runner"
!git config --global user.email "action@github.com"

# --- 4. Run the DVC Pipeline ---
print("\n--- Pulling DVC data ---")
!dvc pull --force

print("\n--- Reproducing DVC pipeline ---")
!dvc repro

# --- 5. Push Artifacts and Metrics ---
print("\n--- Pushing DVC artifacts ---")
!dvc push

print("\n--- Committing results back to Git ---")
!git add .
!git commit -m "CI: Automated run from Kaggle GPU"
!git push

print("\n--- Workflow Finished ---")